In [1]:
# database.py
import sqlite3
import datetime

DATABASE_NAME = 'ponto.db' # Nome do seu arquivo de banco de dados SQLite

def get_conn():
    """Retorna uma conexão com o banco de dados."""
    return sqlite3.connect(DATABASE_NAME)

def init_db():
    """Inicializa as tabelas do banco de dados se elas não existirem."""
    conn = get_conn()
    cursor = conn.cursor()

    # Tabela para armazenar os estagiários
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS estagiarios (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT NOT NULL,
        cpf TEXT NOT NULL UNIQUE,
        senha TEXT NOT NULL
    );
    """)

    # Tabela para armazenar os registros de ponto
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS pontos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        estagiario_id INTEGER NOT NULL,
        data TEXT NOT NULL,
        hora TEXT NOT NULL,
        tipo TEXT NOT NULL, -- 'entrada' ou 'saida'
        FOREIGN KEY(estagiario_id) REFERENCES estagiarios(id)
    );
    """)
    conn.commit()
    conn.close()

def add_estagiario(nome, cpf, senha_hash):
    """
    Adiciona um novo estagiário ao banco de dados.
    Retorna o ID do estagiário recém-adicionado ou None se o CPF já existir.
    """
    conn = get_conn()
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO estagiarios (nome, cpf, senha) VALUES (?, ?, ?)",
                       (nome, cpf, senha_hash))
        conn.commit()
        return cursor.lastrowid
    except sqlite3.IntegrityError:
        # CPF já existe, retorna None para indicar falha
        return None
    finally:
        conn.close()

def get_estagiario_by_cpf(cpf):
    """
    Busca um estagiário pelo CPF.
    Retorna uma tupla com (id, nome, cpf, senha_hashed) ou None se não encontrado.
    """
    conn = get_conn()
    cursor = conn.cursor()
    cursor.execute("SELECT id, nome, cpf, senha FROM estagiarios WHERE cpf = ?", (cpf,))
    estagiario = cursor.fetchone()
    conn.close()
    return estagiario

def record_ponto(estagiario_id, tipo):
    """
    Registra um ponto (entrada/saída) para um estagiário.
    A data e hora são capturadas no momento da execução.
    """
    conn = get_conn()
    cursor = conn.cursor()
    data_atual = datetime.date.today().strftime("%Y-%m-%d")
    hora_atual = datetime.datetime.now().strftime("%H:%M:%S")
    cursor.execute("INSERT INTO pontos (estagiario_id, data, hora, tipo) VALUES (?, ?, ?, ?)",
                   (estagiario_id, data_atual, hora_atual, tipo))
    conn.commit()
    conn.close()

def get_pontos_by_estagiario(estagiario_id):
    """
    Retorna todos os registros de ponto de um estagiário específico,
    ordenados do mais recente para o mais antigo.
    """
    conn = get_conn()
    cursor = conn.cursor()
    cursor.execute("SELECT data, hora, tipo FROM pontos WHERE estagiario_id = ? ORDER BY data DESC, hora DESC",
                   (estagiario_id,))
    pontos = cursor.fetchall()
    conn.close()
    return pontos

def get_all_pontos():
    """
    Retorna todos os registros de ponto de todos os estagiários,
    juntando informações do estagiário (nome, CPF). Útil para um painel de administração.
    """
    conn = get_conn()
    cursor = conn.cursor()
    cursor.execute("""
        SELECT e.nome, e.cpf, p.data, p.hora, p.tipo
        FROM pontos p
        JOIN estagiarios e ON p.estagiario_id = e.id
        ORDER BY p.data DESC, p.hora DESC
    """)
    pontos = cursor.fetchall()
    conn.close()
    return pontos

In [2]:
import sqlite3

# Cria e conecta ao banco de dados SQLite
def connect():
    return sqlite3.connect('banco_estagiarios.db')

# Inicializa o banco e as tabelas
def init_db():
    conn = connect()
    cursor = conn.cursor()
    
    # Tabela de estagiários
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS estagiarios (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nome TEXT NOT NULL,
            cpf TEXT NOT NULL UNIQUE,
            senha TEXT NOT NULL
        )
    ''')

    # Tabela de registros de ponto
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS pontos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            estagiario_id INTEGER NOT NULL,
            data TEXT NOT NULL,
            hora TEXT NOT NULL,
            tipo TEXT NOT NULL,
            FOREIGN KEY(estagiario_id) REFERENCES estagiarios(id)
        )
    ''')

    conn.commit()
    conn.close()

# Adiciona novo estagiário
def add_estagiario(nome, cpf, senha_hashed):
    try:
        conn = connect()
        cursor = conn.cursor()
        cursor.execute('INSERT INTO estagiarios (nome, cpf, senha) VALUES (?, ?, ?)', (nome, cpf, senha_hashed))
        conn.commit()
        return cursor.lastrowid
    except sqlite3.IntegrityError:
        return None
    finally:
        conn.close()

# Busca estagiário pelo CPF
def get_estagiario_by_cpf(cpf):
    conn = connect()
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM estagiarios WHERE cpf = ?', (cpf,))
    result = cursor.fetchone()
    conn.close()
    return result

# Registra entrada ou saída
from datetime import datetime
def record_ponto(estagiario_id, tipo):
    data = datetime.now().strftime('%Y-%m-%d')
    hora = datetime.now().strftime('%H:%M:%S')
    conn = connect()
    cursor = conn.cursor()
    cursor.execute('INSERT INTO pontos (estagiario_id, data, hora, tipo) VALUES (?, ?, ?, ?)',
                   (estagiario_id, data, hora, tipo))
    conn.commit()
    conn.close()

# Retorna pontos do estagiário
def get_pontos_by_estagiario(estagiario_id):
    conn = connect()
    cursor = conn.cursor()
    cursor.execute('SELECT data, hora, tipo FROM pontos WHERE estagiario_id = ? ORDER BY data DESC, hora DESC',
                   (estagiario_id,))
    result = cursor.fetchall()
    conn.close()
    return result

# Retorna todos os pontos com nome e cpf
def get_all_pontos():
    conn = connect()
    cursor = conn.cursor()
    cursor.execute('''
        SELECT e.nome, e.cpf, p.data, p.hora, p.tipo
        FROM pontos p
        JOIN estagiarios e ON p.estagiario_id = e.id
        ORDER BY p.data DESC, p.hora DESC
    ''')
    result = cursor.fetchall()
    conn.close()
    return result


In [3]:
from IPython.core.display import HTML

HTML("""
<link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">

<style>
  .container {
    margin-top: 20px;
    background-color: #f0f0f0;
    padding: 10px;
    border-radius: 8px;
  }
  .navbar {
    background-color: #03264d !important;
  }
  .navbar-brand, .nav-link, .navbar-text {
    color: #fff !important;
  }
</style>

<nav class="navbar navbar-expand-lg">
  <div class="container-fluid">
    <a class="navbar-brand" href="#">TJRS - Sistema de Ponto</a>
    <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#navbarNav" aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation">
      <span class="navbar-toggler-icon"></span>
    </button>
    <div class="collapse navbar-collapse" id="navbarNav">
      <ul class="navbar-nav me-auto mb-2 mb-lg-0">
        <li class="nav-item"><a class="nav-link" href="#">Início</a></li>
        <li class="nav-item"><a class="nav-link" href="#">Sair</a></li>
      </ul>
    </div>
  </div>
</nav>

<div class="container">
  <h2>Bem-vindo(a) ao Sistema de Ponto do TJRS</h2>
  <p>Este conteúdo é exibido dentro do Jupyter Notebook.</p>
</div>

<script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/js/bootstrap.bundle.min.js"></script>
""")


In [6]:
import sqlite3
from datetime import datetime

def connect():
    return sqlite3.connect('banco_estagiarios.db')

def init_db():
    conn = connect()
    cursor = conn.cursor()
    
    # Tabela de estagiários
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS estagiarios (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nome TEXT NOT NULL,
            cpf TEXT NOT NULL UNIQUE,
            senha TEXT NOT NULL
        )
    ''')

    # Tabela de pontos
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS pontos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            estagiario_id INTEGER NOT NULL,
            data TEXT NOT NULL,
            hora TEXT NOT NULL,
            tipo TEXT NOT NULL,
            FOREIGN KEY(estagiario_id) REFERENCES estagiarios(id)
        )
    ''')

    conn.commit()
    conn.close()

def add_estagiario(nome, cpf, senha_hashed):
    try:
        conn = connect()
        cursor = conn.cursor()
        cursor.execute('INSERT INTO estagiarios (nome, cpf, senha) VALUES (?, ?, ?)', (nome, cpf, senha_hashed))
        conn.commit()
        return cursor.lastrowid
    except sqlite3.IntegrityError:
        return None
    finally:
        conn.close()

def get_estagiario_by_cpf(cpf):
    conn = connect()
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM estagiarios WHERE cpf = ?', (cpf,))
    result = cursor.fetchone()
    conn.close()
    return result

def record_ponto(estagiario_id, tipo):
    data = datetime.now().strftime('%Y-%m-%d')
    hora = datetime.now().strftime('%H:%M:%S')
    conn = connect()
    cursor = conn.cursor()
    cursor.execute('INSERT INTO pontos (estagiario_id, data, hora, tipo) VALUES (?, ?, ?, ?)',
                   (estagiario_id, data, hora, tipo))
    conn.commit()
    conn.close()

def get_pontos_by_estagiario(estagiario_id):
    conn = connect()
    cursor = conn.cursor()
    cursor.execute('SELECT data, hora, tipo FROM pontos WHERE estagiario_id = ? ORDER BY data DESC, hora DESC',
                   (estagiario_id,))
    result = cursor.fetchall()
    conn.close()
    return result

def get_all_pontos():
    conn = connect()
    cursor = conn.cursor()
    cursor.execute('''
        SELECT e.nome, e.cpf, p.data, p.hora, p.tipo
        FROM pontos p
        JOIN estagiarios e ON p.estagiario_id = e.id
        ORDER BY p.data DESC, p.hora DESC
    ''')
    result = cursor.fetchall()
    conn.close()
    return result


In [7]:
{% extends "base.html" %}

{% block title %}Meu Ponto - Sistema de Ponto TJRS{% endblock %}

{% block content %}
<div class="row">
    <div class="col-md-8">
        <h2>Bem-vindo(a), {{ estagiario_nome }}!</h2>
        <p>Use os botões abaixo para registrar sua entrada ou saída.</p>

        <form action="{{ url_for('registrar_ponto') }}" method="POST" class="d-inline">
            <input type="hidden" name="tipo" value="entrada">
            <button type="submit" class="btn btn-success btn-lg me-2">Registrar Entrada</button>
        </form>
        <form action="{{ url_for('registrar_ponto') }}" method="POST" class="d-inline">
            <input type="hidden" name="tipo" value="saida">
            <button type="submit" class="btn btn-warning btn-lg">Registrar Saída</button>
        </form>

        <h3 class="mt-4">Seu Histórico de Pontos</h3>
        {% if pontos %}
        <table class="table table-striped table-bordered">
            <thead>
                <tr>
                    <th>Data</th>
                    <th>Hora</th>
                    <th>Tipo</th>
                </tr>
            </thead>
            <tbody>
                {% for ponto in pontos %}
                <tr>
                    <td>{{ ponto[0] }}</td>
                    <td>{{ ponto[1] }}</td>
                    <td>{{ ponto[2] | capitalize }}</td> </tr>
                {% endfor %}
            </tbody>
        </table>
        {% else %}
        <p>Nenhum ponto registrado ainda.</p>
        {% endif %}
    </div>
    <div class="col-md-4 text-center">
        <h3>Seu QR Code de Identificação</h3>
        <p>Este QR Code contém seu ID de estagiário e pode ser usado para leitura externa.</p>
        <img src="data:image/png;base64,{{ qr_code_base64 }}" alt="QR Code do Estagiário" class="img-fluid" style="max-width: 200px;">
    </div>
</div>
{% endblock %}

SyntaxError: invalid syntax (4228972322.py, line 1)

In [ ]:
import sqlite3
import os

DB_NAME = 'ponto_estagiarios.db'

def get_conn():
    return sqlite3.connect(DB_NAME)

def init_db():
    """Cria a tabela de estagiários, se ainda não existir."""
    conn = get_conn()
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS estagiarios (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nome TEXT NOT NULL,
            cpf TEXT UNIQUE NOT NULL,
            senha TEXT
        )
    ''')
    conn.commit()
    conn.close()

def inserir_estagiario(nome, cpf, senha):
    """Insere um estagiário no banco (para fins de teste)."""
    conn = get_conn()
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO estagiarios (nome, cpf, senha) VALUES (?, ?, ?)", (nome, cpf, senha))
        conn.commit()
    except sqlite3.IntegrityError:
        print(f"Estagiário com CPF {cpf} já existe.")
    conn.close()


In [ ]:
<link rel="stylesheet" href="{{ url_for('static', filename='css/main.css') }}">

<script src="{{ url_for('static', filename='js/app.js') }}"></script>

<img src="{{ url_for('static', filename='images/tjrs_logo.png') }}" alt="Logo TJRS">

In [ ]:
<!DOCTYPE html>
<html lang="pt-br">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block title %}Sistema de Ponto TJRS{% endblock %}</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        body { background-color: #f8f9fa; }
        .navbar { background-color: #03264d !important; }
        .navbar-brand, .nav-link, .navbar-text { color: #fff !important; }
        .container { margin-top: 20px; }
    </style>
</head>
<body>
<nav class="navbar navbar-expand-lg">
  <div class="container-fluid">
    <a class="navbar-brand" href="#">TJRS - Sistema de Ponto</a>
    <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#navbarNav" aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation">
      <span class="navbar-toggler-icon"></span>
    </button>
    <div class="collapse navbar-collapse" id="navbarNav">
      <ul class="navbar-nav me-auto mb-2 mb-lg-0">
        <li class="nav-item"><a class="nav-link" href="{{ url_for('ponto_page') }}">Início</a></li>
        {% if 'estagiario_id' in session %}
        <li class="nav-item"><a class="nav-link" href="{{ url_for('logout') }}">Sair</a></li>
        {% else %}
        <li class="nav-item"><a class="nav-link" href="{{ url_for('login') }}">Login</a></li>
        <li class="nav-item"><a class="nav-link" href="{{ url_for('register') }}">Registrar</a></li>
        {% endif %}
      </ul>
    </div>
  </div>
</nav>
<div class="container mt-4">
    {% block content %}{% endblock %}
</div>
<script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/js/bootstrap.bundle.min.js"></script>
</body>
</html>

In [ ]:
<!DOCTYPE html>
<html lang="pt-br">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block title %}Sistema de Ponto TJRS{% endblock %}</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        body { background-color: #f8f9fa; }
        .navbar { background-color: #03264d !important; } /* Cor azul escura do TJRS */
        .navbar-brand, .nav-link, .navbar-text { color: #fff !important; }
        .container { margin-top: 20px; }
    </style>
</head>
<body>
<nav class="navbar navbar-expand-lg">
  <div class="container-fluid">
    <a class="navbar-brand" href="#">TJRS - Sistema de Ponto</a>
    <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#navbarNav" aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation">
      <span class="navbar-toggler-icon"></span>
    </button>
    <div class="collapse navbar-collapse" id="navbarNav">
      <ul class="navbar-nav me-auto mb-2 mb-lg-0">
        <li class="nav-item"><a class="nav-link" href="{{ url_for('ponto_page') }}">Início</a></li>
        {% if 'estagiario_id' in session %}
        <li class="nav-item"><a class="nav-link" href="{{ url_for('logout') }}">Sair</a></li>
        {% else %}
        <li class="nav-item"><a class="nav-link" href="{{ url_for('login') }}">Login</a></li>
        <li class="nav-item"><a class="nav-link" href="{{ url_for('register') }}">Registrar</a></li>
        {% endif %}
      </ul>
    </div>
  </div>
</nav>
<div class="container mt-4">
    {% block content %}{% endblock %}
</div>
<script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/js/bootstrap.bundle.min.js"></script>
</body>
</html>



In [ ]:
import database # Importa as funções do seu arquivo database.py

# app.py
from flask import Flask, render_template, request, redirect, url_for, session, jsonify
from werkzeug.security import generate_password_hash, check_password_hash
import database # Importa as funções do seu arquivo database.py
import qrcode
import base64
from io import BytesIO

app = Flask(__name__)
# MUDE esta chave secreta para uma string longa, aleatória e complexa em um ambiente de produção!
app.secret_key = 'uma_chave_secreta_muito_segura_e_complexa_aqui_1234567890' 

# Esta função é executada antes de cada requisição para garantir que o DB esteja inicializado
@app.before_request
def initialize_database():
    database.init_db()

# Rota da página inicial (redireciona para login ou página de ponto)
@app.route('/')
def index():
    if 'estagiario_id' in session:
        return redirect(url_for('ponto_page'))
    return render_template('login.html')

# Rotas de Login
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        cpf = request.form['cpf']
        senha = request.form['senha']

        estagiario = database.get_estagiario_by_cpf(cpf)

        # Verifica se o estagiário existe e se a senha está correta
        if estagiario and check_password_hash(estagiario[3], senha): # estagiario[3] é a senha hashed no DB
            session['estagiario_id'] = estagiario[0] # Armazena o ID na sessão
            session['estagiario_nome'] = estagiario[1] # Armazena o nome na sessão
            return redirect(url_for('ponto_page'))
        else:
            return render_template('login.html', message='CPF ou senha incorretos.')
    return render_template('login.html')

# Rotas de Registro de Estagiário
@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        nome = request.form['nome']
        cpf = request.form['cpf']
        senha = request.form['senha']

        # Hashing da senha para segurança antes de salvar no DB
        senha_hashed = generate_password_hash(senha)

        estagiario_id = database.add_estagiario(nome, cpf, senha_hashed)
        if estagiario_id:
            return render_template('login.html', message='Cadastro realizado com sucesso! Faça o login.')
        else:
            return render_template('register.html', message='CPF já cadastrado.')
    return render_template('register.html')

# Rota da Página de Ponto (após o login)
@app.route('/ponto')
def ponto_page():
    # Redireciona para o login se não houver estagiário na sessão
    if 'estagiario_id' not in session:
        return redirect(url_for('login'))
    
    estagiario_id = session['estagiario_id']
    estagiario_nome = session['estagiario_nome']
    
    # Gerar QR Code para o ID do estagiário logado
    # Este QR Code pode ser usado para um sistema de leitura externo
    qr_data = f"ESTAGIARIO_ID:{estagiario_id}"
    img = qrcode.make(qr_data)
    
    # Converte a imagem do QR Code para base64 para ser exibida no HTML
    buffer = BytesIO()
    img.save(buffer, format="PNG")
    qr_code_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')

    # Busca o histórico de pontos do estagiário
    pontos = database.get_pontos_by_estagiario(estagiario_id)
    return render_template('ponto.html', 
                           estagiario_nome=estagiario_nome, 
                           pontos=pontos,
                           qr_code_base64=qr_code_base64)

# Rota para Registrar Ponto (usada pelos botões na página de ponto)
@app.route('/registrar_ponto', methods=['POST'])
def registrar_ponto():
    if 'estagiario_id' not in session:
        # Se não estiver logado, retorna um erro JSON ou redireciona
        return jsonify({"message": "Não autenticado"}), 401
    
    estagiario_id = session['estagiario_id']
    tipo = request.form['tipo'] # Recebe 'entrada' ou 'saida' do formulário

    if tipo not in ['entrada', 'saida']:
        return jsonify({"message": "Tipo de ponto inválido"}), 400

    database.record_ponto(estagiario_id, tipo)
    # Redireciona de volta para a página de ponto para ver o histórico atualizado
    return redirect(url_for('ponto_page'))

# Rota de Logout
@app.route('/logout')
def logout():
    # Remove as informações do estagiário da sessão
    session.pop('estagiario_id', None)
    session.pop('estagiario_nome', None)
    return redirect(url_for('login'))

# --- API Endpoints (opcionais, para integração com outros sistemas ou testes) ---
# Você pode usar estes endpoints com ferramentas como Postman ou o Jupyter Notebook.

@app.route('/api/register', methods=['POST'])
def api_register():
    data = request.get_json()
    nome = data.get('nome')
    cpf = data.get('cpf')
    senha = data.get('senha')

    if not all([nome, cpf, senha]):
        return jsonify({"message": "Dados incompletos"}), 400
    
    senha_hashed = generate_password_hash(senha)
    estagiario_id = database.add_estagiario(nome, cpf, senha_hashed)

    if estagiario_id:
        return jsonify({"message": "Estagiário cadastrado com sucesso!", "id": estagiario_id}), 201
    else:
        return jsonify({"message": "CPF já cadastrado"}), 409

@app.route('/api/login', methods=['POST'])
def api_login():
    data = request.get_json()
    cpf = data.get('cpf')
    senha = data.get('senha')

    estagiario = database.get_estagiario_by_cpf(cpf)
    if estagiario and check_password_hash(estagiario[3], senha):
        return jsonify({"message": "Login bem-sucedido!", "id": estagiario[0], "nome": estagiario[1]}), 200
    else:
        return jsonify({"message": "CPF ou senha inválidos"}), 401

@app.route('/api/registrar_ponto', methods=['POST'])
def api_registrar_ponto():
    data = request.get_json()
    estagiario_id = data.get('estagiario_id')
    tipo = data.get('tipo')

    if not all([estagiario_id, tipo]) or tipo not in ['entrada', 'saida']:
        return jsonify({"message": "Dados inválidos"}), 400

    database.record_ponto(estagiario_id, tipo)
    return jsonify({"message": f"Ponto de {tipo} registrado com sucesso!"}), 200

@app.route('/api/meus_pontos/<int:estagiario_id>', methods=['GET'])
def api_meus_pontos(estagiario_id):
    pontos = database.get_pontos_by_estagiario(estagiario_id)
    # Formata os pontos para retornar como JSON
    pontos_formatados = [{"data": p[0], "hora": p[1], "tipo": p[2]} for p in pontos]
    return jsonify(pontos_formatados), 200

@app.route('/api/todos_pontos', methods=['GET'])
def api_todos_pontos():
    pontos = database.get_all_pontos()
    pontos_formatados = [{"nome": p[0], "cpf": p[1], "data": p[2], "hora": p[3], "tipo": p[4]} for p in pontos]
    return jsonify(pontos_formatados), 200


if __name__ == '__main__':
    # Inicializa o banco de dados e as tabelas ao iniciar o script
    database.init_db()
    # Roda o servidor Flask. debug=True é ótimo para desenvolvimento (reinicia ao salvar),
    # mas deve ser False em produção.
    app.run(debug=True)

In [ ]:
from IPython.core.display import HTML
HTML("""
<style>
  .container {
    margin-top: 20px;
    background-color: #f0f0f0;
    padding: 10px;
    border-radius: 8px;
  }
</style>
""")